<a href="https://colab.research.google.com/github/Marouen07/MVA_Deep_Learning_DQN/blob/master/DQN_project_MVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
pip install sk-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
import random

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act as defined  is called epsilon-greedy policy \

$\epsilon$-greedy policy:
$$
\pi(s) = \begin{cases}
\max_a Q(s,a) & \text{w.p.} 1- \epsilon\\
\text{random action} & \text{w.p.} \epsilon
\end{cases}
$$
Epsilon Here is essential in order to make the agent do some exploration : if we simply act greedily the agent would have no reason to do any exploration if it causes a lower reward although exploration is essential in order to discover the environnement and learn an optimal policy. 




***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` defines the reward associated to moving to every cell. The array ```position``` defines where the agent is on the board. It has values -1 on thee extra boarders where the agent can not go, 0 where it can go and 1 on its current position.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        
    def learned_act(self, s):
        return np.random.randint(0,self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state=env.reset()
        done=False 
        win=0
        lose=0
        while not done:
          state, reward, done=env.act(agent.learned_act(state))
          if reward > 0:
            win = win + reward
          if reward < 0:
            lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
random_agent = RandomAgent()

test(random_agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 15.5/12.0. Average score (3.5)
Win/lose count 14.0/12.0. Average score (2.75)
Win/lose count 5.5/7.0. Average score (1.3333333333333333)
Win/lose count 12.5/12.0. Average score (1.125)
Win/lose count 9.5/16.0. Average score (-0.4)
Win/lose count 6.0/8.0. Average score (-0.6666666666666666)
Win/lose count 12.0/8.0. Average score (0.0)
Win/lose count 9.0/5.0. Average score (0.5)
Win/lose count 5.5/8.0. Average score (0.16666666666666666)
Win/lose count 9.5/14.0. Average score (-0.3)
Final score: -0.3


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




The two equations are The bellman equation and the bellman optimal equation 
\begin{align*}
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{0\leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a,a_t=\pi(s_t)].\\
&=E_{p^{\pi}}[r(s,a)+\gamma \sum_{1\leq t}\gamma^{t-1}r(s_{t},a_{t})]|s_{0}=s,a_{0}=a,a_t=\pi(s_t)] \\
&=E_{p^{\pi}}[r(s,a)+\gamma \sum_{0\leq t}\gamma^{t}r(s_{t+1},a_{t+1})]|s_{0}=s,a_{0}=a,a_t=\pi(s_t)] \\
&=E_{p^{\pi}}[r(s,a)+\gamma \sum_{s',a'}\left( p(s1=s',a1=a') E[ \sum_{0\leq t}\gamma^{t}r(s_{t+1},a_{t+1})]|s_{1}=s',a_{1}=a',a_t=\pi(s_t)] \right)|s_{0}=s,a_{0}=a,a_t=\pi(s_t)] \\
&=r(s,a)+\gamma \sum_{s',a'}\left( p(s_1=s',a_1=a'|s_{0}=s,a_{0}=a) E[ \sum_{0\leq t}\gamma^{t}r(s_{t+1},a_{t+1})]|s_{1}=s',a_{1}=a',a_t=\pi(s_t)] \right) \\
&=r(s,a)+\gamma \sum_{s',a'} p(s_1=s',a_1=a'|s_{0}=s,a_{0}=a) Q^\pi(s',a') \\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}


The second equation is obtained exactly the same way 


\begin{align}
Q^{*}(s,a) &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= \max_{\pi}r(s,a) + \gamma \sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )Q^{\pi}(s',a')\\
&= r(s,a) + \gamma \max_{\pi} \sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )Q^{\pi}(s',a') \\
&= r(s,a) + \gamma \sum_{s'}(p(s_{1}=s'|s_0 = s, a_0 = a )\max_{(a',\pi')} Q^{\pi'}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') \\ 
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

The proposed objective function is based on this equality, it's also known as Temporal Difference.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
from collections import namedtuple
Transition = namedtuple('Transition',
                        ('state', 'next_state', 'action', 'reward','game_over'))
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, *args):
      n=len(self.memory)
      if n>=self.max_memory:
        del self.memory[np.random.randint(n)]     
      self.memory.append(Transition(*args))
    def random_access(self,size):
        return random.sample(self.memory, size)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 50 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch import optim
#Model For Q-Function
class Model(nn.Module):
  def __init__(self, n_state):
    super(Model, self).__init__()
    self.n_state = n_state
    self.net = nn.Sequential(
        nn.Linear(5*5*n_state, out_features=32),
        nn.ReLU(),
        nn.Linear(in_features=32, out_features=4),
    )
  def forward(self, state):
    x=state.view(state.size(0),-1)
    #print(state.shape,x.shape)
    return self.net(x)
  def predict(self, x):
      return self(x).max(1)[0].detach() #Detach so we don't compute gradient 


In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        self.grid_size = grid_size

        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        if len(s.shape)==3:
          Q_s=self.model(torch.from_numpy(s).unsqueeze(0).to(device))
          best_action=torch.argmax(Q_s[0])
          return best_action
        else :
          #This will be used in The DQ mimick part 
          #We need to get the set of optimal Actions for Expert DQN
          Q_sa=self.model(s.to(device))
          return torch.argmax(Q_sa,dim=-1)
        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):

        # Two steps: first memorize the states, second learn from the pool
        s_, n_s_, a_, r_, game_over_=torch.from_numpy(s_).unsqueeze(0),torch.from_numpy(n_s_).unsqueeze(0),torch.tensor([a_]),torch.tensor([r_]),torch.tensor([game_over_])
        self.memory.remember(s_, n_s_, a_, r_, game_over_)
        if len(self.memory.memory)<self.batch_size:
          return
        #Getting batch from Replay Buffer
        transitions=self.memory.random_access(self.batch_size)
        batch = Transition(*zip(*transitions))
        state_batch = torch.cat(batch.state).to(device)
        next_batch=torch.cat(batch.next_state).to(device)
        action_batch = torch.cat(batch.action).to(device)
        reward_batch = torch.cat(batch.reward).to(device)
        over_batch=torch.cat(batch.game_over).to(device)
        #Updating Params of NN 
        Q_sa=self.model(state_batch).gather(dim=1,index=action_batch.unsqueeze(1))
        Expected_Q=self.model.predict(next_batch)
        target_q = torch.where(over_batch,reward_batch.double(),(Expected_Q * self.discount) + reward_batch).unsqueeze(1)
        #Clipping Target
        target_q=target_q.clamp(-3,3)
        loss = F.mse_loss(Q_sa, target_q)
        self.optimizer.zero_grad()
        loss.backward()

        #Clipping The gradient 
        for param in self.model.parameters():
            param.grad.data.clamp_(-3, 3)
        self.optimizer.step()
        return loss

    def save(self,name_weights='model.h5',name_model='model.json'):
        torch.save(self.model.state_dict(),name_weights)
        torch.save(self.model,name_model)

            
    def load(self,name_weights='model.h5',name_model='model.json'):
        self.model.load_state_dict(torch.load(name_weights))

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        self.model = Model(self.n_state).double().to(device)
        self.optimizer=optim.Adam(self.model.parameters(),lr=lr)

In [14]:
device=torch.device('cuda') 
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
epochs_train=101
train(agent_fc, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train100.mp4'))

Epoch 000/101 | Loss 0.0487 | Win/lose count 2.5/4.0 (-1.5)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 001/101 | Loss 0.0098 | Win/lose count 0.5/3.0 (-2.5)
Epoch 002/101 | Loss 0.0089 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/101 | Loss 0.0109 | Win/lose count 7.0/3.0 (4.0)
Epoch 004/101 | Loss 0.0047 | Win/lose count 3.0/2.0 (1.0)
Epoch 005/101 | Loss 0.1883 | Win/lose count 4.0/2.0 (2.0)
Epoch 006/101 | Loss 0.0313 | Win/lose count 3.5/3.0 (0.5)
Epoch 007/101 | Loss 0.2796 | Win/lose count 4.0/2.0 (2.0)
Epoch 008/101 | Loss 0.2930 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/101 | Loss 0.0212 | Win/lose count 3.0/1.0 (2.0)
Epoch 010/101 | Loss 0.0077 | Win/lose count 4.0/1.0 (3.0)
Epoch 011/101 | Loss 0.0123 | Win/lose count 8.5/3.0 (5.5)
Epoch 012/101 | Loss 0.2420 | Win/lose count 4.5/1.0 (3.5)
Epoch 013/101 | Loss 0.0068 | Win/lose count 9.0/2.0 (7.0)
Epoch 014/101 | Loss 0.3172 | Win/lose count 11.0/2.0 (9.0)
Epoch 015/101 | Loss 0.0099 | Win/lose count 2.5/3.0 (-0.5)
Epoch 016/101 | Loss 0.0145 | Win/lose count 16.5/3.0 (13.5)
Epoch 017/101 | Loss 0.1000 | Win/lose count 2.0/3

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class Model_conv(nn.Module):
  def __init__(self, n_state):
    super(Model_conv, self).__init__()
    self.n_state = n_state
    self.conv=nn.Sequential(nn.Conv2d(n_state,32,2),nn.ReLU(),
                            nn.Conv2d(32,32,2),nn.ReLU()
                            ) #for input(5x5x2)output should be (3x3x32)
    self.net = nn.Sequential(nn.Linear(32*3*3,50), nn.ReLU(),nn.Dropout(0.5),
                             nn.Linear(50,out_features=4))

  def forward(self, state):
    state=state.permute(0,3,1,2).contiguous()
    x=self.conv(state)
    x=x.reshape(state.size(0),-1)
    x=x.view(state.size(0),-1)
    return self.net(x)
  def predict(self, x):
      return self(x).max(1)[0].detach() #Detach so we don't compute gradient 
      
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        self.model = Model_conv(self.n_state).double().to(device)
        self.optimizer=optim.Adam(self.model.parameters(),lr=lr)


In [16]:
device=torch.device('cuda')
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent_cnn,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train100.mp4'))

Epoch 000/101 | Loss 0.1955 | Win/lose count 8.0/10.0 (-2.0)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Model_conv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 001/101 | Loss 0.2297 | Win/lose count 10.5/10.0 (0.5)
Epoch 002/101 | Loss 0.4527 | Win/lose count 9.0/6.0 (3.0)
Epoch 003/101 | Loss 0.3218 | Win/lose count 12.5/5.0 (7.5)
Epoch 004/101 | Loss 0.2619 | Win/lose count 7.0/9.0 (-2.0)
Epoch 005/101 | Loss 0.2758 | Win/lose count 16.0/5.0 (11.0)
Epoch 006/101 | Loss 0.1356 | Win/lose count 11.5/9.0 (2.5)
Epoch 007/101 | Loss 0.2172 | Win/lose count 5.5/6.0 (-0.5)
Epoch 008/101 | Loss 0.2501 | Win/lose count 11.5/5.0 (6.5)
Epoch 009/101 | Loss 0.2271 | Win/lose count 10.0/6.0 (4.0)
Epoch 010/101 | Loss 0.1940 | Win/lose count 12.0/5.0 (7.0)
Epoch 011/101 | Loss 0.1627 | Win/lose count 7.0/6.0 (1.0)
Epoch 012/101 | Loss 0.1350 | Win/lose count 16.0/9.0 (7.0)
Epoch 013/101 | Loss 0.1222 | Win/lose count 14.5/2.0 (12.5)
Epoch 014/101 | Loss 0.1669 | Win/lose count 10.5/3.0 (7.5)
Epoch 015/101 | Loss 0.1646 | Win/lose count 13.0/8.0 (5.0)
Epoch 016/101 | Loss 0.1305 | Win/lose count 12.0/10.0 (2.0)
Epoch 017/101 | Loss 0.1871 | Win/lose

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
"""agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')"""
print('Test of the CNN')
epochs_test=11
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.0/0. Average score (7.0)
Win/lose count 5.5/1.0. Average score (5.75)
Win/lose count 6.5/1.0. Average score (5.666666666666667)
Win/lose count 8.5/3.0. Average score (5.625)
Win/lose count 7.5/0. Average score (6.0)
Win/lose count 7.0/1.0. Average score (6.0)
Win/lose count 9.0/0. Average score (6.428571428571429)
Win/lose count 8.5/3.0. Average score (6.3125)
Win/lose count 6.5/1.0. Average score (6.222222222222222)
Win/lose count 7.0/2.0. Average score (6.1)
Win/lose count 3.5/1.0. Average score (5.7727272727272725)
Final score: 5.7727272727272725
Test of the FC
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 4.0/0. Average score (2.5)
Win/lose count 3.0/0. Average score (2.6666666666666665)
Win/lose count 0.5/0. Average score (2.125)
Win/lose count 2.5/0. Average score (2.2)
Win/lose count 1.0/0. Average score (2.0)
Win/lose count 1.0/0. Average score (1.8571428571428572)
Win/lose count 1.5/0. Average score (1.8125)
Win/lose count 3.0/0. Ave

In [18]:
HTML(display_videos('fc_test10.mp4'))

the CNN performs much better than the fully connected network. And that the difference is even bigger with larger temperatures. 


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay=0.9,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        if e%5==0 :
          agent.set_epsilon(agent.epsilon*decay)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 50 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')



In [0]:
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        #we add an array to track visited cells 
        self.visited = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        #updating Visited Status: 
        self.visited[self.x,self.y]=1
        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.visited.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        #Resetting Visited states 
        self.visited=np.zeros((self.grid_size, self.grid_size))
        self.visited[self.x,self.y]=1

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1),
                        self.visited.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [22]:
#Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_explore = DQN_CNN(size, lr=.001, epsilon = 0.5, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_explore, env, 100, decay=0.9,prefix='cnn_train_explore') 

Epoch 000/100 | Loss 0.5531 | Win/lose count 9.5/8.0 (1.5)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Model_conv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 001/100 | Loss 0.3387 | Win/lose count 11.0/14.0 (-3.0)
Epoch 002/100 | Loss 0.2372 | Win/lose count 6.0/10.0 (-4.0)
Epoch 003/100 | Loss 0.1303 | Win/lose count 6.5/14.0 (-7.5)
Epoch 004/100 | Loss 0.3714 | Win/lose count 9.5/10.0 (-0.5)
Epoch 005/100 | Loss 0.1646 | Win/lose count 3.5/8.0 (-4.5)
Epoch 006/100 | Loss 0.1165 | Win/lose count 13.5/5.0 (8.5)
Epoch 007/100 | Loss 0.3253 | Win/lose count 9.0/12.0 (-3.0)
Epoch 008/100 | Loss 0.2375 | Win/lose count 8.0/14.0 (-6.0)
Epoch 009/100 | Loss 0.3661 | Win/lose count 17.5/12.0 (5.5)
Epoch 010/100 | Loss 0.2154 | Win/lose count 16.0/10.0 (6.0)
Epoch 011/100 | Loss 0.2516 | Win/lose count 12.0/8.0 (4.0)
Epoch 012/100 | Loss 0.2600 | Win/lose count 16.5/7.0 (9.5)
Epoch 013/100 | Loss 0.1600 | Win/lose count 7.0/1.0 (6.0)
Epoch 014/100 | Loss 0.1946 | Win/lose count 16.5/6.0 (10.5)
Epoch 015/100 | Loss 0.1732 | Win/lose count 10.0/6.0 (4.0)
Epoch 016/100 | Loss 0.1071 | Win/lose count 12.5/8.0 (4.5)
Epoch 017/100 | Loss 0.1493 | W

In [23]:
HTML(display_videos('cnn_train_explore50.mp4'))

In [24]:
# Evaluation
test(agent_explore,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 18.5/2.0. Average score (16.5)
Win/lose count 10.0/0. Average score (13.25)
Win/lose count 20.0/7.0. Average score (13.166666666666666)
Win/lose count 14.5/3.0. Average score (12.75)
Win/lose count 11.5/0. Average score (12.5)
Win/lose count 11.0/3.0. Average score (11.75)
Win/lose count 14.0/2.0. Average score (11.785714285714286)
Win/lose count 1.5/1.0. Average score (10.375)
Win/lose count 17.5/2.0. Average score (10.944444444444445)
Win/lose count 8.0/1.0. Average score (10.55)
Win/lose count 17.5/2.0. Average score (11.0)
Final score: 11.0


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In order to mimic an Expert DQN we'll try to train a DQN with the following two objectives objective:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma Q(s',arg\max_{a'} Q^{expert}(s',a'),\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
and 

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q^{expert}(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


In [0]:
class DQN_mim(Agent):
    def __init__(self,expert, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_mim, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        self.grid_size = grid_size

        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        # Batch size when learning
        self.batch_size = batch_size
        self.expert=expert
    def learned_act(self, s):
        Q_s=self.model(torch.from_numpy(s).unsqueeze(0).to(device))
        best_action=torch.argmax(Q_s[0])
        return best_action
    def reinforce(self, s_, n_s_, a_, r_, game_over_):

        # Two steps: first memorize the states, second learn from the pool
        s_, n_s_, a_, r_, game_over_=torch.from_numpy(s_).unsqueeze(0),torch.from_numpy(n_s_).unsqueeze(0),torch.tensor([a_]),torch.tensor([r_]),torch.tensor([game_over_])
        self.memory.remember(s_, n_s_, a_, r_, game_over_)
        if len(self.memory.memory)<self.batch_size:
          return
        #Getting batch from Replay Buffer
        transitions=self.memory.random_access(self.batch_size)
        batch = Transition(*zip(*transitions))
        state_batch = torch.cat(batch.state).to(device)
        next_batch=torch.cat(batch.next_state).to(device)
        action_batch = torch.cat(batch.action).to(device)
        reward_batch = torch.cat(batch.reward).to(device)
        over_batch=torch.cat(batch.game_over).to(device)
        #Updating Params of NN 
        Q_sa=self.model(state_batch).gather(dim=1,index=action_batch.unsqueeze(1)).squeeze()
        #The Difference is that we use the Expert DQN to know which is the best Action 
        Expected_Q=self.expert.model.predict(next_batch).squeeze()
        #Expected_Q=self.model(next_batch).gather(dim=1,index=self.expert.learned_act(next_batch).unsqueeze(1)).squeeze()
        target_q = torch.where(over_batch,reward_batch.double(),(Expected_Q * self.discount) + reward_batch).squeeze()
        
        #Clipping Target
        target_q=target_q.clamp(-3,3)
        #print(Expected_Q.shape,target_q.shape)
        loss = F.mse_loss(Q_sa, target_q)
        self.optimizer.zero_grad()
        loss.backward()

        #Clipping The gradient 
        for param in self.model.parameters():
            param.grad.data.clamp_(-3, 3)
        self.optimizer.step()
        return loss

    def save(self,name_weights='model.h5',name_model='model.json'):
        torch.save(self.model.state_dict(),name_weights)
        torch.save(self.model,name_model)

            
    def load(self,name_weights='model.h5',name_model='model.json'):
        self.model.load_state_dict(torch.load(name_weights))

class DQN_CNN_mim(DQN_mim):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_mim, self).__init__(*args,**kwargs)
        self.model = Model_conv(self.n_state).double().to(device)
        self.optimizer=optim.Adam(self.model.parameters(),lr=lr)

In [26]:
#Training
agent_mim = DQN_CNN_mim(agent_explore,size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train(agent_mim, env, 100,prefix='cnn_train_explore')

Epoch 000/100 | Loss 0.4817 | Win/lose count 6.0/9.0 (-3.0)


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Model_conv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 001/100 | Loss 0.3844 | Win/lose count 9.0/13.0 (-4.0)
Epoch 002/100 | Loss 0.3381 | Win/lose count 5.5/10.0 (-4.5)
Epoch 003/100 | Loss 0.4779 | Win/lose count 10.5/14.0 (-3.5)
Epoch 004/100 | Loss 0.4735 | Win/lose count 10.5/14.0 (-3.5)
Epoch 005/100 | Loss 0.3335 | Win/lose count 6.5/8.0 (-1.5)
Epoch 006/100 | Loss 0.2589 | Win/lose count 13.5/15.0 (-1.5)
Epoch 007/100 | Loss 0.3032 | Win/lose count 7.5/12.0 (-4.5)
Epoch 008/100 | Loss 0.2691 | Win/lose count 13.5/7.0 (6.5)
Epoch 009/100 | Loss 0.2124 | Win/lose count 7.0/14.0 (-7.0)
Epoch 010/100 | Loss 0.2470 | Win/lose count 12.0/5.0 (7.0)
Epoch 011/100 | Loss 0.1699 | Win/lose count 8.0/12.0 (-4.0)
Epoch 012/100 | Loss 0.2381 | Win/lose count 14.0/23.0 (-9.0)
Epoch 013/100 | Loss 0.1738 | Win/lose count 12.0/14.0 (-2.0)
Epoch 014/100 | Loss 0.1374 | Win/lose count 14.0/14.0 (0.0)
Epoch 015/100 | Loss 0.1792 | Win/lose count 12.0/11.0 (1.0)
Epoch 016/100 | Loss 0.1601 | Win/lose count 9.0/7.0 (2.0)
Epoch 017/100 | Loss 0.1

In [27]:
# Evaluation
print('Test of the CNN')
test(agent_explore,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

print('Test of the Mimic CNN')
test(agent_mim,env,epochs_test,prefix='cnn_test_mim')
HTML(display_videos('cnn_test_mim10.mp4'))

Test of the CNN
Win/lose count 6.0/2.0. Average score (4.0)
Win/lose count 15.5/2.0. Average score (8.75)
Win/lose count 7.0/2.0. Average score (7.5)
Win/lose count 18.5/2.0. Average score (9.75)
Win/lose count 18.5/1.0. Average score (11.3)
Win/lose count 5.5/1.0. Average score (10.166666666666666)
Win/lose count 5.5/1.0. Average score (9.357142857142858)
Win/lose count 11.0/1.0. Average score (9.4375)
Win/lose count 22.0/4.0. Average score (10.38888888888889)
Win/lose count 19.0/5.0. Average score (10.75)
Win/lose count 19.5/4.0. Average score (11.181818181818182)
Final score: 11.181818181818182
Test of the Mimic CNN
Win/lose count 11.5/3.0. Average score (8.5)
Win/lose count 7.0/0. Average score (7.75)
Win/lose count 10.0/1.0. Average score (8.166666666666666)
Win/lose count 2.0/0. Average score (6.625)
Win/lose count 7.5/4.0. Average score (6.0)
Win/lose count 11.0/1.0. Average score (6.666666666666667)
Win/lose count 6.0/1.0. Average score (6.428571428571429)
Win/lose count 8.5/2.

The Mimic DQN can't outperform the expert DQN but it's performance is still better than the one without exploration. : The Student can't beat the master 

***